In [164]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

# ['pearl', 'white', 'ggplot', 'solar', 'space']
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

# Statistics Files Per Run
## Barcode Stats

In [178]:
# transition to read this from constance
barcode_stats = glob("/Users/brow015/Downloads/miseq/*/run_stats/barcode_stats.csv")
bs = df_from_csvs(barcode_stats, header=0)
# bs['RunID'] = bs['RunID'].apply(lambda x: x.split("_")[0] + "_" + x.split("-")[-1])
bs['RunID'] = bs['RunID'].apply(lambda x: x.split("_")[0])
bs['RunID'] = pd.to_datetime(bs['RunID'])
bs.reset_index(inplace=True)
bs.head(3)

,index,RunID,OnTarget,OffTarget,Unmatched,GiniCoefficient
0,0,2015-01-21,0.021433,0.685864,0.292704,0.451497
1,0,2015-03-02,0.041888,0.677877,0.280235,0.293522
2,0,2015-04-13,0.708665,0.000661,0.290674,0.363067


## Sample Stats

In [179]:
sample_stats = glob("/Users/brow015/Downloads/miseq/*/run_stats/sample_stats.csv")
ss = df_from_csvs(sample_stats, header=0)
# ss['run_id'] = ss['run_id'].apply(lambda x: x.split("_")[0] + "_" + x.split("-")[-1])
ss['run_id'] = ss['run_id'].apply(lambda x: x.split("_")[0])
ss['run_id'] = pd.to_datetime(ss['run_id'])
ss['paired_reads'] = ss['individual_reads'] / 2
ss['unjoined_pairs'] = ss['paired_reads'] - ss['joined_pairs']
ss.reset_index(inplace=True)
print("Observations plus the controls: ", len(ss.values))
ss.head(3)

Observations plus the controls:  3722


,index,sample,run_id,individual_reads,joined_pairs,contaminated_reads,paired_reads,unjoined_pairs
0,0,s7163C,2015-01-21,4,0,1,2,2
1,1,s8110A,2015-01-21,0,NaN,NaN,0,NaN
2,2,s7246C,2015-01-21,0,NaN,NaN,0,NaN


In [180]:
# remove the control samples from the stats tables
# the names vary greatly across experiments - need to standardize
controls = ['standard', 'xylanomonas', 'synecho','cytophaga', 'shewanella', 'subtilis', 'saccharophagus',
            'flavobacterium', 'metallosphaera', 'phiX', 'syn7002', 'std1', 'std2', 'std3', 'std4', 'std5', 'std6',
            'stds1', 'stds2', 'stds3', 'stds4', 'stds5', 'stds6', 'Standard', 'MR-1', 'MR1', 'blank', 'Blank']
ss_sub = ss[ss['sample'].str.contains("|".join(controls)) == False]
print("Minus the controls: ", len(ss_sub.values))

Minus the controls:  3608


In [181]:
# read totals across runs minus the controls
totes = ss_sub.groupby(by=['run_id'])['paired_reads'].sum()
totes = totes.reset_index()
totes.head(3)

,run_id,paired_reads
0,2015-01-21,301491
1,2015-03-02,540755
2,2015-04-13,9218595


# Demultiplexing Efficiency
Since we are supplying a reference with all known barcodes, we can determine how many reads map to their respective barcode target, how many hit other, previously used barcodes, and the number that are unmatched.

In [186]:
bs_sub = bs[['RunID', 'OnTarget', 'OffTarget', 'Unmatched']].set_index('RunID')
bs_sub.iplot(kind='area', fill=True, title='Demultiplexing Efficiency')

# On-Target Read Yield Across Runs
The totals include the controls, but do not count reads that have a mismatch in their barcode sequence.

In [183]:
df = totes.set_index('run_id')
df.iplot(kind='bar', title='Raw Read Yield')

# Distribution of Reads Across Barcodes
Using the Gini Coefficient, we can get a sense of how evenly the reads were distributed across samples per run. A low value indicates more even distribution.

In [184]:
df = bs[['RunID', 'GiniCoefficient']].set_index('RunID')
df.iplot(kind='bar', title='On-Target Gini Coefficient (lower is better)')

# Paired-Read Totals Per Sample

In [185]:
df = ss_sub.pivot_table('paired_reads', 'sample', 'run_id')
df.iplot(kind='box', title='Read Pairs Per Sample')

#Joining 16S Data
Reads are joined using `fastq-join -p 240 -m 2`, meaning we require 240 bp overlap with a maximum of 2 mismatches allowed.

In [188]:
df = ss_sub.pivot_table('joined_pairs', 'sample', 'run_id')
df.iplot(kind='box', title='Joined Pairs Per Sample')

#Cross-Contamination
Reads are mapped to positive control, 16S sequences using BWA. We require 95% overlap of mapped bases matching at 99% identity.

In [189]:
df = ss_sub.pivot_table('contaminated_reads', 'sample', 'run_id')
df.iplot(kind='box', title='Contaminated Reads Per Sample')